In [ ]:
# API to get data from Goodreads
from goodreads import client
import pandas as pd
import numpy as np
import requests
import json 
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [ ]:
#instructions on API requests for GOODREADS:
# https://www.goodreads.com/api/index#shelves.list

# key and password to download the info
gc = client.GoodreadsClient('wxwrc6aLfRoMX3Ivr784A','rFT6Ytzh5TRBNcWnAYTdWY1wU5U27fQ6tEegWiSM5M')

# user = gc.user(42889636)
# user.shelves
# # wasnt able to figure out how to unpack this bound method

In [ ]:
#creating functions to return userid's and their readbooks + rating

def createuserIDs(num_users):
    i = 0
    mylist = []
    while i < num_users:
        x = randint(1,99999999)
        if x not in mylist:
            mylist.append(x)
        i+= 1
    return mylist

# removing this usercheck, as its only creating more GET requests and overloading the system
# def createuserlist(list_users,userIDs):
#     for i in userIDs:
#         try:
#             user = gc.user(i)
#             list_users.append((i,user.name))
#             #sleep 1, as they accept max 1 request per second
#             sleep(1)
#         # adding except: pass, as some users are no longer active
#         except:
#             pass
#     return list_users

def createshelve_read(user):
    userID = []
    books = []
    ratings = []
    try:
        url = 'https://www.goodreads.com/review/list/' + str(user) +'.xml?key=wxwrc6aLfRoMX3Ivr784A&v=2&per_page=200&shelf=read'
        html = requests.get(url).content
        soup = BeautifulSoup(html, "lxml")
        for element in soup.find_all('title_without_series'):
            books.append(element.text)
        for element in soup.find_all('rating'):
            ratings.append(element.text)
        #sleep 1, as they accept max 1 request per second
        sleep(1)
        for i in range(len(books)):
            userID.append(user)
    # adding except: pass, as some users are private and you cannot download their info
    except:
        pass
    return userID, books, ratings

In [ ]:
# define some random userIDs
userIDs = createuserIDs(3000)

#adding myself :-)
userIDs.append(42889636)

print(len(userIDs))

In [ ]:
# # my user: 42889636
# # ann: 75206708

# # getting the list of books they have read + rating of said book
# Storing in different lists to run per batches as many times it freezes / blocks
# we may have less users than in user list as some users are either unactive of private

# getting all the books + ratings for said users
alldata = []

for i in userIDs[:500]:
    result = createshelve_read(i)
    alldata.append(result)
len(alldata)


In [ ]:
for i in userIDs[500:1000]:
    result = createshelve_read(i)
    alldata.append(result)
len(alldata)


In [ ]:
for i in userIDs[1000:1500]:
    result = createshelve_read(i)
    alldata.append(result)    
len(alldata)


In [ ]:
for i in userIDs[1500:2000]:
    result = createshelve_read(i)
    alldata.append(result)
len(alldata)


In [ ]:
for i in userIDs[2000:2500]:
    result = createshelve_read(i)
    alldata.append(result)
len(alldata)


In [ ]:
for i in userIDs[2500:]:
    result = createshelve_read(i)
    alldata.append(result)
len(alldata)


In [ ]:
# putting it all in one data frame

userID = [i[0] for i in alldata]
books = [i[1] for i in alldata]
ratings = [i[2] for i in alldata]

flat_userID = [item for sublist in userID for item in sublist]
flat_books = [item for sublist in books for item in sublist]
flat_ratings = [item for sublist in ratings for item in sublist]

df = pd.DataFrame({
    'userid' : flat_userID,
    'book' : flat_books,
    'rating' : flat_ratings,
})

df.head()

In [ ]:
df.to_csv('goodreads.csv',index=False)